In [75]:
library(tidyverse)

# Custom package
library(rutils)

In [76]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [77]:
dset_idx <- 1

# Load and filter survival data

In [78]:
event_code <- list("Alive" = 0, "Dead" = 1)
covariate_cols_no_figo <- c("age_at_diagnosis", "race", "ethnicity")
covariate_cols <- c("figo_stage", covariate_cols_no_figo)
dep_cols <- c("vital_status", "survival_time")

In [79]:
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


In [80]:
filtered_survival_df <- survival_df %>%
    decode_figo_stage(to = "c") %>%
    dplyr::select(sample_name, figo_stage, race, ethnicity, age_at_diagnosis) %>% # make sure using same samples as classification models
    dplyr::filter(rowSums(is.na(.)) == 0) %>%
    dplyr::select(sample_name, figo_stage)

# Load normalized matrisome count data

In [81]:
norm_matrisome_counts_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/norm_matrisome_counts.tsv")
norm_survival_counts_t_df <- read_tsv(norm_matrisome_counts_path) %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name)) %>%
    transpose_df("geneID", "sample_name")

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


In [82]:
joined_df <- filtered_survival_df %>%
    inner_join(norm_survival_counts_t_df, by = "sample_name")

In [83]:
# Some genes contain the '-' symbol, which affects formulae
colnames(joined_df) <- gsub("-", "_", colnames(joined_df))

In [84]:
gene_names <- colnames(joined_df[-c(1:2)])
waov_df <- tibble(geneID = c(), pval = c())

In [85]:
for (g in gene_names) {
    formula_str <- paste0(g, " ~ figo_stage")
    waov_res <- oneway.test(as.formula(formula_str), data = joined_df)
    # Welch one-way ANOVA (doesn't assume equal variance for each group)
    waov_df <- waov_df %>%
        bind_rows(tibble(geneID = g, pval = waov_res$p.value))
}
waov_df <- waov_df %>%
    # restore original gene names (no longer care about formula)
    dplyr::mutate(geneID = gsub("_", "-", geneID)) %>%
    dplyr::mutate(padj = p.adjust(pval, method = "BH"))

In [86]:
write_tsv(waov_df, paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))